In [14]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import sklearn.metrics as mt
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')



In [2]:
pip install tensorflow pandas sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [15]:
import pandas as pd

# Open the original CSV file for reading and a new CSV file for writing
with open('data.csv', 'r') as infile, open('cleaned.csv', 'w') as outfile:
    for line in infile:
        # Your data cleaning logic goes here
        # For example, you can check for and fix incomplete strings
        cleaned_line = line.replace('"incomplete data', '"complete data')

        # Write the cleaned line to the new file
        outfile.write(cleaned_line)

# Now you can read the cleaned data using pandas
df = pd.read_csv('cleaned.csv')


In [16]:
import pandas as pd

# Attempt to read the CSV file with error handling
try:
    df1 = pd.read_csv('data.csv', encoding='utf-8', error_bad_lines=False)
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    df1 = None

if df1 is not None:
    # Select columns of interest
    df = df1[['Review Text', 'Rating', 'Class Name', 'Age']]
    df1.head()
else:
    print("CSV file couldn't be loaded due to parsing errors.")


In [17]:
df1.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1082.0,18.0,Versatile Wonderful Cute Love Love Cute,Dress looks like it's made of,3.0,0.0,10.0,General,Jackets,Outerwear
1,827.0,45.0,"Loved, but returned Flattering Great Super cute",I'm torn! Stylish Love Wonderful,4.0,1.0,16.0,General,Dresses,Dresses
2,474.0,48.0,Not for the very petite,Super cute and cozy Great Stylish,5.0,1.0,9.0,General,Intimates,Outerwear
3,791.0,34.0,Love this Super cute Super cute Love Cute,My favorite buy Super cute Wonderful,1.0,0.0,13.0,General,Jackets,Pants
4,667.0,37.0,Flattering Great Cute Super cute Love Stylish,Love this Versatile Love Cute Love,4.0,1.0,11.0,General,Intimates,Lounge


In [18]:

df1.describe()

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,43486.000000,43486.000000,43486.000000,43486.000000,43486.000000
mean,795.547740,41.232075,3.645955,0.674470,5.976406
std,293.608318,12.503245,1.391089,0.468578,6.951038
min,0.000000,18.000000,1.000000,0.000000,0.000000
25%,610.000000,32.000000,3.000000,0.000000,0.000000
50%,867.000000,40.000000,4.000000,1.000000,3.000000
75%,1045.000000,51.000000,5.000000,1.000000,11.000000
max,1205.000000,99.000000,5.000000,1.000000,122.000000


In [19]:
df1.shape

(43487, 10)

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


# Now you can read the cleaned data using pandas
df = pd.read_csv('cleaned.csv')

# fill NA values by space
df['Review Text'] = df['Review Text'].fillna('')

# Tokenize the text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Review Text'])
word_index = tokenizer.word_index

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['Review Text'])

# Pad sequences to ensure uniform input size
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

# Convert sentiment labels to binary (0 for negative, 1 for positive)
df['Sentiment'] = df['Rating'] >= 4
labels = df['Sentiment'].astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential([
    Embedding(5000, 16, input_length=100),  # Increased embedding dimension
    LSTM(32),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with increased batch size
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
1088/1088 [==============================] - 58s 51ms/step - loss: 0.6459 - accuracy: 0.6248 - val_loss: 0.6723 - val_accuracy: 0.5982
Epoch 2/10
1088/1088 [==============================] - 55s 51ms/step - loss: 0.6312 - accuracy: 0.6478 - val_loss: 0.6673 - val_accuracy: 0.5862
Epoch 3/10
1088/1088 [==============================] - 55s 50ms/step - loss: 0.6083 - accuracy: 0.6693 - val_loss: 0.5985 - val_accuracy: 0.6817
Epoch 4/10
1088/1088 [==============================] - 56s 52ms/step - loss: 0.6165 - accuracy: 0.6400 - val_loss: 0.6621 - val_accuracy: 0.5553
Epoch 5/10
1088/1088 [==============================] - 57s 52ms/step - loss: 0.5363 - accuracy: 0.6990 - val_loss: 0.5147 - val_accuracy: 0.7206
Epoch 6/10
1088/1088 [==============================] - 55s 51ms/step - loss: 0.4806 - accuracy: 0.7399 - val_loss: 0.5206 - val_accuracy: 0.6857
Epoch 7/10
1088/1088 [==============================] - 53s 49ms/step - loss: 0.4540 - accuracy: 0.7554 - val_loss: 0.5100 -

In [23]:
# Predict the classes for the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import classification_report
# Print the classification report
print(classification_report(y_test, y_pred))


272/272 [==============================] - 4s 14ms/step
              precision    recall  f1-score   support

           0       0.63      0.90      0.74      3495
           1       0.91      0.64      0.75      5203

    accuracy                           0.74      8698
   macro avg       0.77      0.77      0.74      8698
weighted avg       0.79      0.74      0.75      8698

